# NOTEBOOK 1 - Coordinate systems

#### A quick introduction to the coordinate systems dq, alpha beta and abc

This is the first notebook in a series providing examples and code to test concepts. Still a work in progress, if there is anything missing or you have some general feedback please contact Sjur Føyen at foyen.sjur@ntnu.no 

Outline of notebook:
- Example 1.1 shows a pratical application of the alpha beta and dq transforms


Code implemented by; Varg Førland. Date; 23.01.2025

Last update; 23.01.2025

GitHub Copilot used for assistance in coding.

In [1]:
# IMPORTS. RUN FIRST!
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from matplotlib.patches import Arc
#more

## Example 1.1 - alpha beta and dq-transformation


#### The abc and $\alpha \beta$ coordinate frames
abc coordinates is the three phase values which can be measured. If these are balanced, i.e. the sum of the three is zero, then the abc coordinates can be transformed to alpha-beta coordinates. The alpha-beta representation is a vector in a 2D plane. This vector has a magnitude and an angle. 
Bear in mind this does not change the value of the voltage or current, only describes it in a different frame of reference, which is more easy to work with. This tranformation is called the Clarke Transform, and can be described mathematically as

$$
\begin{bmatrix}
I_{\alpha} \\
I_{\beta}
\end{bmatrix}
=
\frac{2}{3}
\begin{bmatrix}
1 & -\frac{1}{2} & -\frac{1}{2} \\
0 & \frac{\sqrt{3}}{2} & -\frac{\sqrt{3}}{2}
\end{bmatrix}
\begin{bmatrix}
I_a \\
I_b \\
I_c
\end{bmatrix}
$$

#### The dq coordinate frame

Now that we have a simpler way of representing three phase values, we consider another transformation, the dq transformation. 

As seen in the we can see in the example below, alpha beta vectors rotate with time, making them complicated to use in calculations.
We now want to represent the values as constant values. If we introduce a new coordinate system, 
where the axes rotates together with the vector, it can be seen as a constant value in that coordinate system.
This is the reasoning behind the dq coordinates.

The tranformation is called the Park transform, and is described as:

$$
\begin{bmatrix}
I_d \\
I_q
\end{bmatrix}
=
\begin{bmatrix}
\cos(\theta) & \sin(\theta) \\
-\sin(\theta) & \cos(\theta)
\end{bmatrix}
\begin{bmatrix}
I_{\alpha} \\
I_{\beta}
\end{bmatrix}
$$


In [6]:
# Function to change from abc to alpha-beta coordinates
def abc_to_alphabeta(abc: np.ndarray) -> np.ndarray:
    """ Change from abc to alpha-beta coordinates. This is done by the rotational matrix shown below. 
     The variable "abc" is a numpy array with the three phase values, i.e. a vector [Ia, Ib, Ic] or [Va, Vb, Vc]. """
    T_alpha_beta = (2/3) * np.array([
        [1, -0.5, -0.5],
        [0, np.sqrt(3)/2, -np.sqrt(3)/2]
    ])
    return T_alpha_beta @ abc # Matrix multiplication for the Clarke transformation matrix

def alphabeta_to_dq(alpha_beta : np.ndarray, theta : float) -> np.ndarray:
    """ Change from alpha-beta to dq coordinates. This is done by the rotational matrix shown below. 
     The variable "alpha_beta" is a numpy array with the two phase values, i.e. a vector [I_alpha, I_beta] or [V_alpha, V_beta]. 
     The variable "theta" is the angle of rotation. """
    T_dq = np.array([
        [np.cos(theta), np.sin(theta)],
        [-np.sin(theta), np.cos(theta)]
    ])
    return T_dq @ alpha_beta # Matrix multiplication for the Park transformation matrix

### PLOT EXAMPLE
Consider a measured line current. From this we get three values, the a, b and c magnitudes of the current. With this we have the abc representation of the current as Iabc = [Ia, Ib, Ic]. This is mathematically hard to work with, so we want to represent the three phase values, as ONE vector in the alpha-beta coordinates. 

Run the code, and experiment with change the time constant. Consider the following questions:
- How does the alpha beta representation compare to the abc representation
- How does the vectors change with time? Does the dq representation change?
- What are the advantages of the different representations?

In [3]:
# PLOT THE CURRENTS; RUN AFTER THE CODE ABOVE

#Chose voltage magnitude, current magintude and phase difference:
Vmag = 1    #pu
Imag = 0.8  #pu
Iphase_diff = -20 * np.pi / 180 # radians

# Plotting
def plot_function(t, Vm, Im, Iphase_diff):
    wt = 2 * np.pi * 50 * t
    theta = wt

    # ABC phase voltages at given time
    Va = Vm * np.cos(wt)
    Vb = Vm * np.cos(wt - 2 * np.pi / 3)
    Vc = Vm * np.cos(wt + 2 * np.pi / 3)
    Vabc = np.array([[Va], [Vb], [Vc]])

    # ABC phase currents at given time; -20 degrees compared to the voltages and 0.8 magnitude
    phase_diff = -20 * np.pi/180
    Ia = Im * np.cos(wt + Iphase_diff)
    Ib = Im * np.cos(wt - 2 * np.pi / 3 + Iphase_diff)
    Ic = Im * np.cos(wt + 2 * np.pi / 3 + Iphase_diff)
    Iabc = np.array([[Ia],[Ib],[Ic]])

    V_alpha_beta = abc_to_alphabeta(Vabc)
    I_alpha_beta = abc_to_alphabeta(Iabc)
    Vdq = alphabeta_to_dq(V_alpha_beta, theta)
    Idq = alphabeta_to_dq(I_alpha_beta, theta)
    fig, (ax2, ax1, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    # Add d and q axis vectors
    ax1.quiver(0, 0, 1.35*np.cos(wt), 1.35*np.sin(wt), angles='xy', scale_units='xy', scale=1, color='grey', width=0.003)
    ax1.quiver(0, 0, -1.35*np.sin(wt), 1.35*np.cos(wt), angles='xy', scale_units='xy', scale=1, color='grey', width=0.003)

    # Annotate d and q axes
    ax1.text(1.35*np.cos(wt) + 0.15, 1.35*np.sin(wt), 'd', fontsize=12, color='grey', ha='center')
    ax1.text(-1.35*np.sin(wt) + 0.15, 1.35*np.cos(wt), 'q', fontsize=12, color='grey', ha='center')

    # Plot alpha-beta voltages and currents
    ax1.axhline(0, color='black', linewidth=0.5)
    ax1.axvline(0, color='black', linewidth=0.5)
    ax1.quiver(0, 0, V_alpha_beta[0], V_alpha_beta[1], angles='xy', scale_units='xy', scale=1, color='forestgreen', label=r'$V_{\alpha\beta}$', width=0.005)
    ax1.quiver(0, 0, I_alpha_beta[0], I_alpha_beta[1], angles='xy', scale_units='xy', scale=1, color='cyan', label=r'$I_{\alpha\beta}$', width=0.005)

    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_aspect('equal', adjustable='box')
    ax1.legend()
    ax1.set_title(r'$\alpha \beta$ Voltages and Currents')
    ax1.grid(True)

    # Calculate and annotate the angle of the voltage vector
    angle = np.arctan2(V_alpha_beta[1], V_alpha_beta[0])
    angle_deg = np.rad2deg(angle).item()  # Convert to scalar

    # Annotate the angle at the base of the voltage vector
    ax1.text(0.2, 0.2, f'{angle_deg:.2f}°', fontsize=12, color='black')


    # Draw an arc to represent the angle
    arc = Arc((0, 0), 0.5, 0.5, angle=0, theta1=0, theta2=angle_deg, color='black', linestyle='--')
    ax1.add_patch(arc)

    # Annotate axis as alpha and beta
    ax1.set_xlabel(r'$\alpha$')
    ax1.set_ylabel(r'$\beta$')

    # Plot abc voltages and currents
    ax2.axhline(0, color='black', linewidth=0.5)
    ax2.axvline(0, color='black', linewidth=0.5)
    ax2.quiver(0, 0, np.cos(wt), np.sin(wt), angles='xy', scale_units='xy', scale=1, color='g', label='Va', width=0.005)
    ax2.quiver(0, 0, np.cos(wt + 4 * np.pi / 3), np.sin(wt + 4 * np.pi / 3), angles='xy', scale_units='xy', scale=1, color='r', label='Vb', width=0.005)
    ax2.quiver(0, 0, np.cos(wt + 2 * np.pi / 3), np.sin(wt + 2 * np.pi / 3), angles='xy', scale_units='xy', scale=1, color='b', label='Vc', width=0.005)
    ax2.quiver(0, 0, Ia, .8 * np.sin(wt + phase_diff), angles='xy', scale_units='xy', scale=1, color='cyan', label='Ia', width=0.005)
    ax2.quiver(0, 0, Ib, .8 * np.sin(wt - 2 * np.pi / 3 + phase_diff), angles='xy', scale_units='xy', scale=1, color='magenta', label='Ib', width=0.005)
    ax2.quiver(0, 0, Ic, .8 * np.sin(wt + 2 * np.pi / 3 + phase_diff), angles='xy', scale_units='xy', scale=1, color='yellow', label='Ic', width=0.005)
    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_aspect('equal', adjustable='box')
    ax2.legend()
    ax2.set_title(r'$abc$ Voltages and Currents')
    ax2.grid(True)

    # Plot dq voltages and currents
    ax3.axhline(0, color='black', linewidth=0.5)
    ax3.axvline(0, color='black', linewidth=0.5)
    ax3.quiver(0, 0, Vdq[0, 0], Vdq[1, 0], angles='xy', scale_units='xy', scale=1, color='forestgreen', label=r'$V_{dq}$', width=0.005)
    ax3.quiver(0, 0, Idq[0, 0], Idq[1, 0], angles='xy', scale_units='xy', scale=1, color='cyan', label=r'$I_{dq}$', width=0.005)
    ax3.set_xlim(-1.5, 1.5)
    ax3.set_ylim(-1.5, 1.5)
    ax3.set_aspect('equal', adjustable='box')
    ax3.legend()
    ax3.set_title(r'$dq$ Coordinates')
    ax3.set_xlabel('d')
    ax3.set_ylabel('q')
    ax3.grid(True)

    plt.suptitle(r'$abc$ and $\alpha \beta$ Voltages and Currents at $t = {:.4f}$'.format(t), fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# Create the slider
time_slider = FloatSlider(min=0, max=0.02, step=0.0005, value=0)

# Use interact to update the plot with the slider
interact(lambda t: plot_function(t, Vmag, Imag, Iphase_diff), t=time_slider)

interactive(children=(FloatSlider(value=0.0, description='t', max=0.02, step=0.0005), Output()), _dom_classes=…

<function __main__.<lambda>(t)>

#### Numerical example

Below is a numerical example if you wish to look closer at the code.

In [4]:
# Time variable
t = .008  # You can change this value to any desired time

wt = 2 * np.pi * 50 * t

# ABC phase voltages at given time
Va = 1.0 * np.cos(wt)
Vb = 1.0 * np.cos(wt - 2 * np.pi / 3)
Vc = 1.0 * np.cos(wt + 2 * np.pi / 3)

# ABC phase currents at given time; -20 degrees compared to the voltages and 0.8 magnitude
phase_diff = -20 * np.pi/180
Ia = .8 * np.cos(wt + phase_diff)
Ib = .8 * np.cos(wt - 2 * np.pi / 3 + phase_diff)
Ic = .8 * np.cos(wt + 2 * np.pi / 3 + phase_diff)

abc_voltages = np.array([[Va], 
                         [Vb], 
                         [Vc]])

abc_currents = np.array([[Ia], 
                         [Ib], 
                         [Ic]])

# Check if the three-phase voltages are balanced
if not np.allclose(np.sum(abc_voltages), 0):
    raise ValueError("The three-phase voltages are not balanced. The sum of Va, Vb, and Vc must be zero.")

# Check if the three-phase currents are balanced
if not np.allclose(np.sum(abc_currents), 0):
    raise ValueError("The three-phase currents are not balanced. The sum of Ia, Ib, and Ic must be zero.")



# Transform to alpha-beta frame
V_alpha_beta = abc_to_alphabeta(abc_voltages)

I_alpha_beta = abc_to_alphabeta(abc_currents)

V_alpha = V_alpha_beta[0, 0]
V_beta = V_alpha_beta[1, 0]
I_alpha = I_alpha_beta[0, 0]
I_beta = I_alpha_beta[1, 0]

# Transform to dq frame

# The angle of rotation is the same as the angle of the voltage phasor i.e. the dq frame is referenced to the voltage phasor
theta = wt
Vdq = alphabeta_to_dq(V_alpha_beta, theta)
Idq = alphabeta_to_dq(I_alpha_beta, theta)



In [5]:
# PRINT CELL, RUN TO SEE NUMERICAL VALUES OF THE VOLTAGES AND CURRENTS

# Prepare data for the table
data = [
    ["Va", Va, "Ia", Ia],
    ["Vb", Vb, "Ib", Ib],
    ["Vc", Vc, "Ic", Ic],
    ["V_alpha", V_alpha, "I_alpha", I_alpha],
    ["V_beta", V_beta, "I_beta", I_beta],
    ["Vd", Vdq[0, 0], "Id", Idq[0, 0]],
    ["Vq", Vdq[1, 0], "Iq", Idq[1, 0]]
]

# Define headers
headers = ["Voltage", "Value", "Current", "Value"]

# Print the table
print(f"{headers[0]:<10} {headers[1]:<10} {headers[2]:<10} {headers[3]:<10}")
print("-" * 40)
for row in data:
    print(f"{row[0]:<10} {row[1]:<10.3f} {row[2]:<10} {row[3]:<10.3f}")
    print("-" * 40)

# Calculate magnitude and angle of the alpha-beta vector
magnitude_v = np.sqrt(V_alpha**2 + V_beta**2)
angle_v = np.arctan2(V_beta, V_alpha)
angle_deg_v = np.rad2deg(angle_v)

magnitude_i = np.sqrt(I_alpha**2 + I_beta**2)
angle_i = np.arctan2(I_beta, I_alpha)
angle_deg_i = np.rad2deg(angle_i)

# Print phasor representations
phasor_data = [
    ["Alpha-Beta Voltage", magnitude_v, angle_deg_v],
    ["Alpha-Beta Current", magnitude_i, angle_deg_i],
    ["dq Voltage", np.sqrt(Vdq[0, 0]**2 + Vdq[1, 0]**2), np.rad2deg(np.arctan2(Vdq[1, 0], Vdq[0, 0]))],
    ["dq Current", np.sqrt(Idq[0, 0]**2 + Idq[1, 0]**2), np.rad2deg(np.arctan2(Idq[1, 0], Idq[0, 0]))]
]

phasor_headers = ["Phasor", "Magnitude", "Angle (degrees)"]

print("\nPhasor Representations:")
print(f"{phasor_headers[0]:<20} {phasor_headers[1]:<10} {phasor_headers[2]:<15}")
print("-" * 45)
for row in phasor_data:
    print(f"{row[0]:<20} {row[1]:<10.3f} {row[2]:<15.3f}")
    print("-" * 45)

Voltage    Value      Current    Value     
----------------------------------------
Va         -0.809     Ia         -0.447    
----------------------------------------
Vb         0.914      Ib         0.798     
----------------------------------------
Vc         -0.105     Ic         -0.351    
----------------------------------------
V_alpha    -0.809     I_alpha    -0.447    
----------------------------------------
V_beta     0.588      I_beta     0.663     
----------------------------------------
Vd         1.000      Id         0.752     
----------------------------------------
Vq         -0.000     Iq         -0.274    
----------------------------------------

Phasor Representations:
Phasor               Magnitude  Angle (degrees)
---------------------------------------------
Alpha-Beta Voltage   1.000      144.000        
---------------------------------------------
Alpha-Beta Current   0.800      124.000        
---------------------------------------------
dq Voltage   